In [28]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from IPython.display import display
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
from numpy import nan
display(HTML("<style>"
    + "#notebook { padding-top:0px; } " ""
    + ".container { width:100%; } "
    + ".end_space { min-height:0px; } "
    + "</style>"))
# import colors


def highlight_max(sett):
    if sett.dtype == 'O':
        s = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
    else:
        s = sett
    is_max = s == s.max()
    
    return ['font-weight: bold' if v else '' for v in is_max]

def highlight_group(sett):
    try:
        if sett.dtype == 'O':
            oboe = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
        else:
            oboe = sett
        is_max = oboe.to_frame().apply(lambda x: oboe[oboe.index.get_level_values(0) == x.name[0]].max() == x.item(), axis=1)
        return ['font-weight: bold' if v else '' for v in is_max]
    except:
        return ['' for _ in sett]

def highlight_group_min(sett):
    try:
        if sett.dtype == 'O':
            oboe = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
        else:
            oboe = sett
        is_max = oboe.to_frame().apply(lambda x: oboe[oboe.index.get_level_values(0) == x.name[0]].min() == x.item(), axis=1)
        return ['font-weight: bold' if v else '' for v in is_max]
    except:
        return ['' for _ in sett]

    
def highlight_min(sett):
    if sett.dtype == 'O':
        s = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
    else:
        s = sett
    is_max = s == s.min()
    
    return ['font-weight: bold' if v else '' for v in is_max]

def b_g(sett, cmap='PuBu', low=0, high=0):
    if sett.dtype == 'O':
        s = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
    else:
        s = sett
    a = s
    rng = a.max() - a.min()
    norm = colors.Normalize(a.min() - (rng * low),
                        a.max() + (rng * high))
    normed = norm(a.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

def method_to_df(apath, setting, dataset, method):
    last = ' '.join([setting, dataset, method])
    with open(apath + '%s/%s/' % (setting, dataset) + method +'/logs.pyd', 'r') as f:
        lines = f.read().splitlines()
    o = pd.DataFrame.from_records([eval(x.replace('nan', 'float(\'nan\')')) for x in lines])
    
    o = o.rename(columns={'cs_size':'buffer_size'})
    if 'buffer_size' not in o:
        o['buffer_size'] = 0
    if 'job_number' in o.columns:
        del o['job_number']
    o['method'] = method
    return o


req_rep = 1
print_args= False
display_pre = False
howmany_pre = 1
print_other_metrics = False
t_distinct_cols = ['buffer_size', 'method', 'shap_weight', 'shap_all', 'n_epochs']
ignore_cols = []
excluded_methods = []
included_methods = []
excluded_datasets = []

sec_respath = None

# PAMI
respath = '../storage/results/dataset-can-mnist/classes_per_task-1/logs.pyd'#'C:\\Users\\emace\\AImageLab\\SRV-Continual\\results\\mammoth\\results\\'
sec_respath = None #'/home/mbosc/srv-nas/mammoth-master/results/'
third_respath = None #'/home/mbosc/phd/pami/results_buzz/'

respaths = [respath, sec_respath, third_respath]
islist = lambda col: any([type(x) == list for x in col.values])

def aggf(x):
#     if len(x) > 2:
#         return x.sort_values()[1:-1].mean()
#     else:
        return x.mean()

# -------------------



In [115]:
## import file
with open(os.path.join(respath), 'r') as obj:
    file = obj.readlines()
file_parsed = map(eval, file)
df_file = pd.DataFrame(list(file_parsed))

In [124]:
df_file.head()

,seed,logs,dataset,model,optim,lr,joint,batch_size,n_epochs,latent_space,sparse_weight,norm_order,classes_per_task,id,timestamp,auc_final,auc_average,auc_per_task,conf_matrix_per_task
0,3588477890,False,can-mnist,sae,sgd,0.1,False,16,2,32,1.0,1,1,22da4cf3-85b1-47c2-96eb-90da14df1274,2022-01-28 18:13:11.536771,0.505262,0.549148,"{'0': {'[0]': 0.8114526675415593, '[1]': nan, ...","{0: {'0[0]': 76.50629796008674, '1[1]': 147.41..."
1,1939617363,False,can-mnist,sae,sgd,0.1,False,16,2,32,1.0,1,1,44a40b10-0528-4ae3-bf89-d8e6b8a3309f,2022-01-28 18:15:21.385007,0.395129,0.504338,"{'0': {'[0]': 0.5473158853342505, '[1]': nan, ...","{0: {'0[0]': 87.49594212356878, '1[1]': 162.67..."
2,1225029300,False,can-mnist,sae,sgd,0.1,False,16,2,32,1.0,1,1,b972705f-362b-48ba-9cac-cc8476432218,2022-01-28 18:29:27.464874,0.444863,0.537492,"{'0': {'[0]': 0.5584516833341762, '[1]': nan, ...","{0: {'0[0]': 136.12866878120266, '1[1]': 143.8..."


In [137]:
total_auc = pd.concat([pd.DataFrame.from_dict(sing[1].loc[0,'auc_per_task'], orient='index')]*2)
total_auc.reset_index().groupby('index').mean()

def calc_mean_on_matrix(X):
    total_df = pd.concat([pd.DataFrame.from_dict(x, orient='index') for x in X])
    return total_df.reset_index().groupby('index').mean()

In [134]:
def calc_mean_pm_std(x):
    return f'{x.mean().round(3)}+-{x.std().round(3)}'

In [135]:
# Converting links to html tags
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [147]:
df_elab = df_file.copy()

common_keys=['dataset','model','optim','lr','joint','batch_size','n_epochs','latent_space','classes_per_task']
part_keys = {'sae':['sparse_weight','norm_order']}
elaboration_cols = {
    'auc_final':[calc_mean_pm_std, len],
    'auc_average':[calc_mean_pm_std, len],
    'auc_per_task': [calc_mean_on_matrix],
    'conf_matrix_per_task': [calc_mean_on_matrix]
    
}
task_per_task_elaboration_cols = {
    
}
total_list = []
for model, df_model in df_elab.groupby('model'):
    df_model['model'] = model
    total_keys = common_keys+part_keys[model]
    
    df_single = df_model.groupby(total_keys).agg(elaboration_cols)
        
    total_list.append(df_single)

pd.concat(total_list)

auc_final          auc_average                                           auc_per_task                               conf_matrix_per_task
                                                                                                           calc_mean_pm_std len calc_mean_pm_std len                                calc_mean_on_matrix                                calc_mean_on_matrix
dataset   model optim lr  joint batch_size n_epochs latent_space classes_per_task sparse_weight norm_order                                                                                                                                                
can-mnist sae   sgd   0.1 False 16         2        32           1                1.0           1              0.448+-0.055   3      0.53+-0.023   3  [[0.6390734120699954, nan, nan, nan, nan, nan,...  [[100.04363628828605, 151.3144173472917, 195.6...

In [36]:
# gridmap(methods, ['3ea70747-8697-4e76-bc4f-319c6fa312b5', '89f4c672-f16a-4b08-a509-a94c82a6ebbd', '31d4cc8c-a727-4665-8643-2212eb519aa6', '26ea2435-d2d4-4988-b37e-d95934301d2f', 'a0626b3b-b34f-4353-8453-4abd769c130c', '9ae21984-b77f-4d12-be42-c4929c4d4125', '1cc0e452-8112-47ac-b939-f9ad5ae5edc6', '86d1bc60-d263-46be-b90f-ff9d9a30173a', '606653f0-b606-426c-9e8f-7770e8dad36a', '6ef54be8-cedf-40d7-a7c7-cda396b45d1c', '918ff2d4-b0d1-456d-96a1-e283e7838827'],
# ['57976485-efee-4aae-95e2-4a2ac112ac3e', 'ff4eb4f9-0e6b-4336-bd66-057a02d2f663', '590d6225-04f7-4ba6-b8c7-0182262ccdf5', '78b3b890-bcc0-4fa6-bc7a-2569f12cee23', '0c875241-6230-41d1-9875-09a2c4c586c0', 'f608773d-99e2-49c9-b1d2-6857966d6977', 'f9a60776-ec0b-47fd-8895-27743c5c9911', '74b41430-89b3-4c1c-a058-9e01bb05480a', '45679ee3-d077-4e5a-83cc-77d4670d27ea', '1e7573db-8821-4cd7-b243-3f4fe9d31553', 'ad3fa7bc-8a5a-4fa6-a2b1-5d3072597e02'])